In [1]:
import nivapy3 as nivapy
import pandas as pd

# Update ICPW "trends" stations for Norway

Norwegian ICPW sites are automatically updates as data are approved by the lab. However, for the "trends" work, we created a separate set of 83 Norwegian sites, which must be updated manually. This notebook exports the relevant data to CSV.

In [2]:
eng = nivapy.da.connect()

Username:  ···
Password:  ········


Connection successful.


In [3]:
# Period of interest
st_dt = '2017-01-01'
end_dt = '2019-12-31'

In [4]:
# Read station details for "trends" sites
stn_df = pd.read_excel(r"../../../all_icpw_sites_may_2020.xlsx", sheet_name="all_icpw_stns")
nor_df = stn_df.query("country == 'Norway'")

# Get the original station codes by removing "Tr18_NO_"
code_list = list(nor_df['station_code'].str[8:])
print(len(nor_df))
nor_df.head()

83


,station_id,station_code,station_name,latitude,longitude,altitude,continent,country,region,group
192,38269,Tr18_NO_1004-13,Store Eitlandsvatn,58.494446,6.735276,394.0,Europe,Norway,SoNord,Trends
193,38310,Tr18_NO_1004-15,Botnevatnet,58.286048,6.490128,51.0,Europe,Norway,SoNord,Trends
194,38306,Tr18_NO_1014-12,Sognevatn,58.316266,7.675424,266.0,Europe,Norway,SoNord,Trends
195,38298,Tr18_NO_1014-25,Drivenesvatn,58.292608,7.930483,174.0,Europe,Norway,SoNord,Trends
196,38292,Tr18_NO_101-605,Holvatn,59.102665,11.525462,164.0,Europe,Norway,SoNord,Trends


In [5]:
# Get all RESA stations
resa_df = nivapy.da.select_resa_stations(eng)
resa_df = resa_df.query("station_code in @code_list")
print(len(resa_df))
resa_df.head()

19711 stations in the RESA database.
83


,station_id,station_code,station_name,latitude,longitude,altitude
96,100,623-603,Breidlivatnet,59.974265,10.146954,635.0
97,101,301-605,Langvann,60.107202,10.757415,342.0
98,102,221-605,Store Lyseren,59.775661,11.773188,229.0
99,103,914-501,Sandvatn,58.685067,8.962033,150.0
100,104,137-501,Ravnsjøen,59.402993,10.998982,82.0


In [6]:
# Get all available parameters
par_df = nivapy.da.select_resa_station_parameters(resa_df, st_dt, end_dt, eng)
par_df.head()

37 parameters available for the selected stations and dates.


,parameter_id,parameter_name,unit
0,3,ALK,mmol/l
1,50,Al,µg/l
2,55,Al/Il,µg/l
3,48,Al/R,µg/l
4,223,As,µg/l


In [7]:
# Get chemistry data
wc_df, dup_df = nivapy.da.select_resa_water_chemistry(resa_df, par_df, st_dt, end_dt, eng, drop_dups=True, lod_flags=True)
wc_df.head()

WARNING
The database contains unexpected duplicate values for some station-date-parameter combinations.
Only the most recent values will be used, but you should check the repeated values are not errors.
The duplicated entries are returned in a separate dataframe.



,station_id,station_code,station_name,sample_date,depth1,depth2,ALK_mmol/l,Al/Il_µg/l,Al/R_µg/l,Al_µg/l,...,SO4_mg/l,SiO2_mg SiO2/l,TOC_mg C/l,TOTN_µg/l N,TOTP_µg/l P,TURB860_FNU,TURB_FNU,Temp_oC,Zn_µg/l,pH_
,,,,,,,,,,,,,,,,,,,,,
0,100,623-603,Breidlivatnet,2017-10-18,0.0,0.0,0.034,92.0,200.0,NaN,...,0.44,NaN,8.4,330.0,10.0,NaN,NaN,NaN,NaN,5.05
1,100,623-603,Breidlivatnet,2018-10-18,0.0,0.0,0.034,66.0,160.0,NaN,...,0.71,NaN,5.4,292.871,8.0,NaN,NaN,NaN,NaN,5.24
2,100,623-603,Breidlivatnet,2019-10-28,0.0,0.0,0.044,97.0,200.0,260.0,...,0.58,1.54,8.5,330.0,8.0,NaN,NaN,NaN,9.4,5.15
3,100,623-603,Breidlivatnet,2019-11-06,0.0,0.0,0.034,120.0,220.0,NaN,...,0.55,NaN,8.3,310.0,9.0,NaN,NaN,NaN,NaN,4.97
4,101,301-605,Langvann,2017-11-03,0.0,0.0,0.063,54.0,79.0,NaN,...,1.06,NaN,5.1,220.0,3.0,NaN,NaN,NaN,NaN,6.01


In [8]:
# Filter data
# Just surface samples
wc_df = wc_df.query("(depth1 < 1) and (depth2 < 1)")

# Ignore mixed samples
wc_df = wc_df.query("depth1 == depth2")

# If still duplicates, choose shallowest
wc_df.sort_values(['depth1'], inplace=True)
wc_df.drop_duplicates(['station_code', 'sample_date'], keep='first', inplace=True)

In [9]:
# Tidy
wc_df['code'] = "Tr18_NO_" + wc_df['station_code']
wc_df.drop(['station_id', 'station_code', 'station_name', 'depth1', 'depth2'], axis='columns', inplace=True)
wc_df.to_csv('out.csv')